In [1]:
import folium
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import GoogleV3
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv() 
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [3]:

base_url = 'https://www.businesslist.com.ng/category/pharmacies/'

# Initialize an empty list to store the results
pharmacies = []

# Loop through pages 1 to 12 (or however many pages there are)
for page_num in range(1, 13):  # Adjust the range as needed
    # Construct the URL for the current page
    url = f"{base_url}{page_num}/city:lagos"
    
    # Fetch the page
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all pharmacy listings on the current page
    for pharmacy in soup.find_all('div', class_=['company with_img g_0','company g_0']):
        # Extract the pharmacy name
        name_tag = pharmacy.find('h4')
        name = name_tag.find('a').text if name_tag else None

        # Extract the address
        address_tag = pharmacy.find('div', class_='address')
        address = address_tag.text.strip() if address_tag else None

        # Add the pharmacy information to the list if both name and address are found
        if name and address:
            pharmacies.append({'name': name, 'address': address})

# Print the results
print(len(pharmacies))

332


In [6]:
# Replace 'YOUR_API_KEY' with your actual Google Maps API key
geolocator = GoogleV3(api_key=GOOGLE_API_KEY,timeout=10)

pharmacy_coordinates = []

# Loop through addresses and get coordinates
for place in pharmacies:
    location = geolocator.geocode(place['address'])
    if location:
        pharmacy_coordinates.append({
            'latitude': location.latitude,
            'longitude': location.longitude,
            'Name': place['name']
        })
        


# Print the results
print(len(pharmacy_coordinates))


ConfigurationError: Since July 2018 Google requires each request to have an API key. Pass a valid `api_key` to GoogleV3 geocoder to fix this error. See https://developers.google.com/maps/documentation/geocoding/usage-and-billing

In [4]:
pharmacy_coordinates_df = pd.DataFrame.from_dict(pharmacy_coordinates)
pharmacy_coordinates_df.to_csv(r'C:\Users\findo\OneDrive\Desktop\Geospace\pharmacy_coordinates.csv')

In [6]:
# Coordinates for the center of lagos
from folium import plugins
lagos_center = [6.465422, 3.406448]  # Approximate geographical center of lagos
# Create the map
lagos_map = folium.Map(location=lagos_center, tiles="OpenStreetMap", zoom_start=9)
for coordinate in pharmacy_coordinates:
    folium.Marker(location=[coordinate['latitude'],coordinate['longitude']],popup=coordinate['Name']).add_to(lagos_map)

lagos_map


In [7]:
lagos_map.save(r'C:\Users\findo\OneDrive\Desktop\Geospace\Pharmacies in Lagos.html')

In [9]:
heat_map_lagos = folium.Map(location=lagos_center, tiles="Cartodb dark_matter", zoom_start=9)
heatmap_data = [[coordinate['latitude'], coordinate['longitude']] for coordinate in pharmacy_coordinates] # Add heatmap layer to the map
plugins.HeatMap(heatmap_data).add_to(heat_map_lagos)
heat_map_lagos

In [ ]:
ma